In [4]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import requests
import re

import warnings 
warnings.filterwarnings('ignore')


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
r=requests.get('https://en.wikipedia.org/wiki/Aviation')
raw_html=r.text
raw_html

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Aviation - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ac110b52-6e87-4a32-9286-974f2c6e5054","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Aviation","wgTitle":"Aviation","wgCurRevisionId":1027140766,"wgRevisionId":1027140766,"wgArticleId":58422,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1: Julian–Gregorian uncertainty","Harv and Sfn no-target errors","Articles with short description","Short description is different from Wikidata","Articles needing additional r

In [8]:
## cleaing up
corpus_html=bs.BeautifulSoup(raw_html)


## extracting paragraphs from html

corpus_paras=corpus_html.find_all('p')
corpus_text=''
##concatenating all the paras

for para in corpus_paras:
    corpus_text+=para.text.lower()
corpus_text


'\naviation is the activities surrounding mechanical flight and the aircraft industry. aircraft includes fixed-wing and rotary-wing types, morphable wings, wing-less lifting bodies, as well as lighter-than-air craft such as hot air balloons and airships.\naviation began in the 18th century with the development of the hot air balloon, an apparatus capable of atmospheric displacement through buoyancy. some of the most significant advancements in aviation technology came with the controlled gliding flying of otto lilienthal in 1896; then a large step in significance came with the construction of the first powered airplane by the wright brothers in the early 1900s. since that time, aviation has been technologically revolutionized by the introduction of the jet which permitted a major form of transport throughout the world.\nthe word aviation was coined by the french writer and former naval officer gabriel la landelle in 1863.[1] he derived the term from the verb avier (an unsuccessful neol

In [9]:
corpus_text

'\naviation is the activities surrounding mechanical flight and the aircraft industry. aircraft includes fixed-wing and rotary-wing types, morphable wings, wing-less lifting bodies, as well as lighter-than-air craft such as hot air balloons and airships.\naviation began in the 18th century with the development of the hot air balloon, an apparatus capable of atmospheric displacement through buoyancy. some of the most significant advancements in aviation technology came with the controlled gliding flying of otto lilienthal in 1896; then a large step in significance came with the construction of the first powered airplane by the wright brothers in the early 1900s. since that time, aviation has been technologically revolutionized by the introduction of the jet which permitted a major form of transport throughout the world.\nthe word aviation was coined by the french writer and former naval officer gabriel la landelle in 1863.[1] he derived the term from the verb avier (an unsuccessful neol

In [10]:
## getting rid of all the empty spaces and special characters
corpus_text=re.sub(r'\[[0-9]*\]',' ',corpus_text) # get rid of number and spaces

corpus_text=re.sub(r'\s+',' ',corpus_text)#for special characters
print(corpus_text)

 aviation is the activities surrounding mechanical flight and the aircraft industry. aircraft includes fixed-wing and rotary-wing types, morphable wings, wing-less lifting bodies, as well as lighter-than-air craft such as hot air balloons and airships. aviation began in the 18th century with the development of the hot air balloon, an apparatus capable of atmospheric displacement through buoyancy. some of the most significant advancements in aviation technology came with the controlled gliding flying of otto lilienthal in 1896; then a large step in significance came with the construction of the first powered airplane by the wright brothers in the early 1900s. since that time, aviation has been technologically revolutionized by the introduction of the jet which permitted a major form of transport throughout the world. the word aviation was coined by the french writer and former naval officer gabriel la landelle in 1863. he derived the term from the verb avier (an unsuccessful neologism f

In [11]:
#reconverting text into sentences and word tokens
corpus_sentences=nltk.sent_tokenize(corpus_text)
corpus_words=nltk.word_tokenize(corpus_text)

In [12]:
corpus_words

['aviation',
 'is',
 'the',
 'activities',
 'surrounding',
 'mechanical',
 'flight',
 'and',
 'the',
 'aircraft',
 'industry',
 '.',
 'aircraft',
 'includes',
 'fixed-wing',
 'and',
 'rotary-wing',
 'types',
 ',',
 'morphable',
 'wings',
 ',',
 'wing-less',
 'lifting',
 'bodies',
 ',',
 'as',
 'well',
 'as',
 'lighter-than-air',
 'craft',
 'such',
 'as',
 'hot',
 'air',
 'balloons',
 'and',
 'airships',
 '.',
 'aviation',
 'began',
 'in',
 'the',
 '18th',
 'century',
 'with',
 'the',
 'development',
 'of',
 'the',
 'hot',
 'air',
 'balloon',
 ',',
 'an',
 'apparatus',
 'capable',
 'of',
 'atmospheric',
 'displacement',
 'through',
 'buoyancy',
 '.',
 'some',
 'of',
 'the',
 'most',
 'significant',
 'advancements',
 'in',
 'aviation',
 'technology',
 'came',
 'with',
 'the',
 'controlled',
 'gliding',
 'flying',
 'of',
 'otto',
 'lilienthal',
 'in',
 '1896',
 ';',
 'then',
 'a',
 'large',
 'step',
 'in',
 'significance',
 'came',
 'with',
 'the',
 'construction',
 'of',
 'the',
 'first'

In [13]:
#1+log(No of docs)/no of docs
#term frequency

In [14]:
greeting_inputs=("hey","good morning","good evening","morning","evening","hi","whatsup")
greeting_outputs=["hey","hey hows you","*nods*","hello, how you doing", "hello","Welcome, I am good"]
def greet_responses(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_outputs)
        


# Preprocessing with punctuation removal and lematizzing

In [15]:
#ran and run , running are in same lemma
#better and good in same lemma
#lemmatizing is get to the root of the word
#stem often createe words that dont have nay meaning
#lemma generate actual words

wn_lemmatizer=nltk.stem.WordNetLemmatizer()
#print(wn_lemmatizer.lemmatize('run'))
def lemmatize_corpus(tokens):    #lemmatizing all the tokens and create a list of lemmatized tokens
    return [wn_lemmatizer.lemmatize(token) for token in tokens]

#dictionary contains ordered punctuation and replace it with none
punct_removal_dict=dict((ord(punctuation),None) for punctuation in string.punctuation)


def get_processed_text(document):  #removing punctuations
    return lemmatize_corpus(nltk.word_tokenize(document.lower().translate(punct_removal_dict)))

#print(g_processed_text("run!!!??)"))

# Language Modeling with tf-idf

In [16]:
def respond(user_input):
    bot_response=''
    corpus_sentences.append(user_input)
    
    #vectorizing the precessed text
    word_vectorizer=TfidfVectorizer(tokenizer=get_processed_text,stop_words='english')
    corpus_word_vectors=word_vectorizer.fit_transform(corpus_sentences)
    
    cos_sim_vectors=cosine_similarity(corpus_word_vectors[-1],corpus_word_vectors)
    similar_response_idx=cos_sim_vectors.argsort()[0][-2]
    
    matched_vector=cos_sim_vectors.flatten()
    matched_vector.sort()
    vector_matched=matched_vector[-2]
    
    if vector_matched==0:
        bot_response=bot_response+'I am sorry, what is it, again?'
        return bot_response
    else:
        bot_response=bot_response+corpus_sentences[similar_response_idx]
        return bot_response
#respond("hi i 2 c")    
    

In [ ]:
chat=True
print('Hello what do you want to learn about Aviation today?')
while(chat==True):
    user_query=input()
    user_query=user_query.lower()
    if user_query !='quit':
        if(user_query=="thanks" or user_query=="thank you"):
            chat=False
            print("AviationBot: You are welcome!")
        else:
            if(greet_responses(user_query)!=None):
                print("AviationBot: "+ greet_responses(user_query))
            else:
                print("AviationBot: ",end="")
                print(respond(user_query))
                corpus_sentences.remove(user_query)
                
    else:
            chat=False
            print("AviationBot: GoodBye!")
            
            
        
    


Hello what do you want to learn about Aviation today?
types of Aviation
AviationBot: each country regulates aviation differently, but general aviation usually falls under different regulations depending on whether it is private or commercial and on the type of equipment involved.
